In [3]:
from Environments.SimulationEnvironment_LOO import SimulationEnvironmentLOO
from Learners.Learner_AdaptiveContextualLinUCBPolicy import AdaptiveContextualLinUCBPolicy
from Utils.Utils import *
import numpy as np
from tqdm.auto import tqdm

In [4]:
dataHouse = pd.read_csv('data/house.csv')
dataHouse

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,70,RL,60.000000,7200,Pave,Reg,Lvl,AllPub,Corner,Gtl,...,0,112,0,0,0,0,8,2009,WD,Normal
1,60,RL,69.331454,18450,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,265,0,0,0,0,0,8,2007,WD,Abnorml
2,70,RM,68.000000,9928,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,28,0,0,0,0,400,6,2009,WD,Normal
3,20,FV,60.000000,7200,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,121,0,0,0,0,0,8,2006,New,Partial
4,120,FV,35.000000,4274,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,116,0,0,0,0,0,11,2007,New,Partial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,20,RL,75.000000,9000,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,8,2009,WD,Abnorml
497,60,RL,69.331454,16698,Pave,IR1,HLS,AllPub,CulDSac,Mod,...,193,0,0,0,0,0,6,2009,WD,Normal
498,60,RL,62.000000,7415,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,75,0,0,0,0,0,4,2008,WD,Normal
499,80,RL,69.331454,14115,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,48,0,0,0,0,0,6,2009,WD,Normal


In [7]:
########################################################################################################################
# Variables to store the rewards and the metrics
########################################################################################################################
curr_policy_reward_per_experiment = []

curr_policy_regret_per_experiment = []

curr_policy_correct_predictions_per_experiment = 0

All_arms_pulled = {}

# At the end of all the experiments I want to plot all the different rewards and regrets for all the combinations
All_experiments_results = {}

########################################################################################################################
# Utils variables
########################################################################################################################

mapping_ArmToMethod = {
    0: "impute_standard",
    1: "impute_mean",
    2: "impute_median",
    3: "impute_random",
    4: "impute_knn",
    5: "impute_mice",
    6: "impute_linear_regression",
    7: "impute_random_forest",
    8: "impute_cmeans"
}

mapping_MethodToArm = {
    "impute_standard": 0,
    "impute_mean": 1,
    "impute_median": 2,
    "impute_random": 3,
    "impute_knn": 4,
    "impute_mice": 5,
    "impute_linear_regression": 6,
    "impute_random_forest": 7,
    "impute_cmeans": 8
}


########################################################################################################################
# Simulation Parameters
########################################################################################################################
percentage_usage_data = 0.1
seeds = [0, 1, 2, 3, 4, 5, 6, 7]
slate_number = 3

num_interactions_in_kb = 500
num_interactions_in_kb = 0
#sim_threshold = 0.5
sim_threshold = 0.9
#topK = 3
topK = 5
#alpha_confidence = 0.5
alpha_confidence = 0.9
#alpha_reward = 0.5
alpha_reward = 0.9

n_arms = 9
n_features = 6
n_experiments = 1
#n_experiments = 2

policy = "LinUCB_KB"

In [8]:
dataset_name_test = 'house'

In [9]:
# Reset variables for this experiment
curr_policy_reward_per_experiment = []

curr_policy_regret_per_experiment = []

curr_policy_relevance_per_experiment = []

opt_per_experiment = []


seed = seeds[0]

# Initialize the simulation environment
env = SimulationEnvironmentLOO(percentage_usage_data, num_interactions_in_kb, alpha_reward, seed, dataset_name_test)
train_data = env.train_dataset
test_data = env.test_data
print(f'Initializing knowledge base with {len(env.transactions_in_kb)} transactions...')
initial_knowledge_base, initial_knowledge_base_table = create_knowledge_base(env.transactions_in_kb)

curr_relevance = []


learner = AdaptiveContextualLinUCBPolicy(n_arms=n_arms, n_features=n_features,
                                         alpha=alpha_confidence,
                                         sim_threshold=sim_threshold, k=topK)
learner.initialize_knowledge_base(initial_knowledge_base, initial_knowledge_base_table)

Initializing knowledge base with 0 transactions...


0it [00:00, ?it/s]

In [10]:

################################################################################################################
# Training
################################################################################################################
print('Training the learners ...')
T_train = len(train_data)
T_train = 100
for t in tqdm(range(T_train)):
    # User and Dataset information retrieval
    interaction = env.get_interaction()

    user_context = env.get_user_context(interaction)
    dataset_profiling = env.get_dataset_profiling(interaction)

    # Create user context
    learner.generate_context(user_context, dataset_profiling)

    # Select an arm
    policy_arm_pulled = learner.pull_slate_arms(slate_number)

    # Obtain the offline interaction
    offline_interaction = env.get_offline_interaction(interaction, policy_arm_pulled[0])

    # Update the knowledge base
    learner.update_knowledge_base_results(offline_interaction, dataset_profiling)
    
    # Obtain the target analysis value
    target_analysis_values = learner.get_target_analysis_value()[policy_arm_pulled[0]]['target_analysis_value']

    # Obtain the reward
    reward = env.get_offline_reward_per_arm(policy_arm_pulled[0], target_analysis_values, offline_interaction)

    # Obtain the reward for each learner
    regret = env.get_regret_per_arm(reward, offline_interaction)

    # Update each learner's state
    learner.update(policy_arm_pulled[0], reward, regret)

Training the learners ...


  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
learner.A

[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.18800000e+03, 2.00827150e+02, 6.74948247e+02,
         5.81314612e+02, 2.29068000e+05],
        [0.00000000e+00, 2.00827150e+02, 5.29325118e+01, 1.36275536e+02,
         1.17162132e+02, 4.34186350e+04],
        [0.00000000e+00, 6.74948247e+02, 1.36275536e+02, 4.60281999e+02,
         3.94581390e+02, 1.43767958e+05],
        [0.00000000e+00, 5.81314612e+02, 1.17162132e+02, 3.94581390e+02,
         3.48489418e+02, 1.23064180e+05],
        [0.00000000e+00, 2.29068000e+05, 4.34186350e+04, 1.43767958e+05,
         1.23064180e+05, 6.02804160e+07]]),
 array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 4.69500000e+03, 3.66588962e+02, 1.27603684e+03,
         1.06798679e+03, 4.10780000e+05],
        [0.00000000e+00, 3.66588962e+02, 5.38032298e+01, 1.40212108e+02,
     

In [6]:
learner.b

[array([0.00000000e+00, 6.70223260e+02, 1.36155330e+02, 4.57818081e+02,
        3.92571485e+02, 1.43688236e+05]),
 array([0.00000000e+00, 1.27642458e+03, 1.40427856e+02, 4.84856873e+02,
        4.05973316e+02, 1.60549927e+05]),
 array([     0.        ,   1206.26201559,    164.14429137,    483.14260603,
           421.57463552, 159972.08941123]),
 array([    0.        ,   758.09294745,    42.84280852,   138.64142548,
          114.51211876, 13327.55140923]),
 array([0.00000000e+00, 1.14212149e+03, 1.24654262e+02, 4.63100483e+02,
        4.02847962e+02, 1.47589866e+05]),
 array([0.00000000e+00, 1.35076038e+03, 1.53708075e+02, 4.89202666e+02,
        4.17276049e+02, 1.65711756e+05]),
 array([    0.        ,   664.99783991,    37.14871447,   121.56077574,
          100.63826503, 10239.17655842]),
 array([     0.        ,   1272.20187108,    153.6680317 ,    445.88146681,
           384.24469749, 136806.87314101]),
 array([    0.        ,   697.66834181,    38.9039127 ,   129.05577389,
    

In [7]:
learner.theta

[array([ 0.00000000e+00, -1.66453378e-03,  4.52679335e-02,  9.61455621e-01,
         9.73572163e-03,  4.44493586e-05]),
 array([ 0.00000000e+00,  2.98200430e-03,  4.03712329e-02,  9.84341807e-01,
        -2.09899353e-02,  3.40449689e-05]),
 array([0.00000000e+00, 3.56299766e-03, 5.64692713e-02, 9.29694452e-01,
        3.07407939e-02, 4.09642437e-05]),
 array([ 0.00000000e+00,  1.09025569e-01,  4.05011998e-02,  4.42525454e-01,
        -3.82677392e-02, -2.39362374e-04]),
 array([ 0.00000000e+00,  2.90657749e-03,  3.21314409e-02,  9.76147870e-01,
        -3.74646966e-03,  2.95528797e-05]),
 array([ 0.00000000e+00,  3.74171066e-03,  4.63397442e-02,  9.83114732e-01,
        -2.44815991e-02,  3.77664600e-05]),
 array([ 0.00000000e+00,  1.19428649e-01,  5.63362110e-02,  3.79970932e-01,
        -3.72992275e-02, -3.19093402e-04]),
 array([0.00000000e+00, 5.12370410e-03, 6.33809843e-02, 9.07537655e-01,
        4.47697460e-02, 4.67091910e-05]),
 array([ 0.00000000e+00,  1.27589152e-01,  5.4175609

In [11]:
################################################################################################################
# Testing Performance Evaluation
################################################################################################################
curr_policy_map_at_1_per_experiment = 0

curr_policy_map_at_3_per_experiment = 0

T_test = len(test_data)
#T_test = 500
i = 0

In [11]:
learner.prob_t

array([1.00320138, 1.00601717, 1.00353563, 0.95773187, 1.0059887 ,
       1.00778278, 0.96014931, 1.00488948, 0.96097288])

In [12]:
test_pull_arms_top1 = np.zeros(n_arms)
test_pull_arms_top3 = np.zeros(n_arms)

In [21]:
interaction = test_data.iloc[i]
interaction

data_object               house_BedroomAbvGr
initial_completeness                0.193362
technique_completeness           impute_mice
ml_algorithm                    RandomForest
final_ml_value                      0.858915
Name: 0, dtype: object

In [22]:
arm_pulled = mapping_MethodToArm[interaction['technique_completeness']]

test_interaction = env.get_test_interaction(interaction["data_object"],
                                                interaction["initial_completeness"],
                                                interaction["ml_algorithm"])
test_interaction

,data_object,initial_completeness,technique_completeness,ml_algorithm,final_ml_value
41288,house_BedroomAbvGr,0.193362,impute_mice,RandomForest,0.858915


In [23]:
user_context = env.get_user_context_test(test_interaction)
dataset_profiling = env.get_dataset_profiling_test(test_interaction)

In [24]:
user_context

array(['classification', 'RandomForest', '0.1933625'], dtype='<U32')

In [25]:
dataset_profiling

,data_object,n_tuples,missing_perc,uniqueness,min,max,mean,median,std,skewness,...,mad,iqr,p_min,p_max,k_min,k_max,s_min,s_max,entropy,density
696,house_BedroomAbvGr,501.0,0.193362,0.017012,0.125,6.0,2.868962,3.0,0.782038,0.267637,...,0.0,1.0,0.00355,0.666625,0.002837,0.591763,0.002987,0.66165,1.138488,2.41095


In [26]:
# Create user context
learner.generate_context(user_context, dataset_profiling)

In [27]:
learner.current_context

[array([ 0.        ,  3.        ,  0.1933625 ,  0.91407162,  0.76038427,
        15.        ]),
 array([ 0.        ,  3.        ,  0.1933625 ,  0.91249439,  0.84159258,
        10.        ]),
 array([ 0.        ,  3.        ,  0.1933625 ,  0.9110994 ,  0.87068955,
        12.        ]),
 array([0.        , 3.        , 0.1933625 , 0.91431654, 0.90130451,
        9.        ]),
 array([ 0.        ,  3.        ,  0.1933625 ,  0.91309761,  0.81246723,
        11.        ]),
 array([ 0.        ,  3.        ,  0.1933625 ,  0.91171384,  0.82990782,
        11.        ]),
 array([0.        , 3.        , 0.1933625 , 0.92021229, 0.8491238 ,
        9.        ]),
 array([ 0.        ,  3.        ,  0.1933625 ,  0.91695492,  0.82026457,
        13.        ]),
 array([ 0.        ,  3.        ,  0.1933625 ,  0.92846602,  0.86924548,
        10.        ])]

In [30]:
# Select an arm for each learner
linucb_arms_kb = learner.pull_slate_arms(slate_number)
linucb_arms_kb

array([4, 3, 7])

In [31]:
learner.prob_t

array([1.55354842, 1.56111471, 1.60057669, 1.61277026, 1.67937255,
       1.51867393, 1.39621444, 1.60558955, 1.45834024])

In [32]:
env.current_interaction_dataframe

,data_object,initial_completeness,technique_completeness,ml_algorithm,final_ml_value
5285,house_BedroomAbvGr,0.193362,impute_random_forest,RandomForest,0.860556
5983,house_BedroomAbvGr,0.193362,impute_knn,RandomForest,0.859721
19024,house_BedroomAbvGr,0.193362,impute_cmeans,RandomForest,0.860883
19578,house_BedroomAbvGr,0.193362,impute_median,RandomForest,0.858535
22745,house_BedroomAbvGr,0.193362,impute_random,RandomForest,0.859534
41288,house_BedroomAbvGr,0.193362,impute_mice,RandomForest,0.858915
42844,house_BedroomAbvGr,0.193362,impute_mean,RandomForest,0.860144
46848,house_BedroomAbvGr,0.193362,impute_linear_regression,RandomForest,0.860790
59035,house_BedroomAbvGr,0.193362,impute_standard,RandomForest,0.858998


In [42]:
# Change interaction to the one that the learner has pulled
interaction.iloc[2] = 'impute_knn'

/var/folders/tt/9v6608x550n51759gpcrffsc0000gn/T/ipykernel_85715/2572735103.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interaction.iloc[2] = 'impute_knn'


In [43]:
interaction

data_object               house_BedroomAbvGr
initial_completeness                0.193362
technique_completeness            impute_knn
ml_algorithm                    RandomForest
final_ml_value                      0.858915
Name: 0, dtype: object

In [44]:
# Simulate the interaction
learner.update_knowledge_base_results(interaction, dataset_profiling)

In [45]:
learner.knowledge_base_table.tail()

,data_object,n_tuples,missing_perc,uniqueness,min,max,mean,median,std,skewness,...,p_min,p_max,k_min,k_max,s_min,s_max,entropy,density,technique_completeness,final_ml_value
96,96_fried_X4,40768.0,0.200000,0.030700,0.000,1.000,0.500250,0.4975,0.288875,-0.000862,...,0.000937,0.020188,0.000612,0.013562,0.000925,0.020188,6.892762,0.000037,impute_random_forest,0.888903
97,97_letter_xegvy,2001.0,0.100387,0.007575,2.000,13.875,8.337262,8.0000,1.550212,0.133375,...,0.005463,0.547600,0.010575,0.466237,0.014500,0.564037,1.797975,0.578188,impute_cmeans,0.842989
98,98_ringnorm_V1,7400.0,0.401363,0.630250,-6603.250,6197.500,212.034875,162.0000,1492.585000,-0.223750,...,0.006237,0.041837,0.005700,0.027700,0.008388,0.041050,7.786537,0.000000,impute_mice,0.910532
99,99_cancer_Bland_Chromatin,683.0,0.297400,0.020875,1.000,10.000,3.402163,3.0000,2.442675,1.125000,...,0.351362,0.753238,0.336650,0.598900,0.390763,0.718087,1.909863,0.845850,impute_knn,0.950018
100,100_house_BedroomAbvGr,501.0,0.193362,0.017012,0.125,6.000,2.868962,3.0000,0.782038,0.267637,...,0.003550,0.666625,0.002837,0.591763,0.002987,0.661650,1.138488,2.410950,impute_knn,0.858915


In [ ]:
# Calculate MAP@K
# map_at_1 = env.calculate_map_at_1(test_interaction)
map_at_1, relevance_rank_at_1 = env.calculate_map_at_1(linucb_arms_kb)
map_at_3, relevance_rank_at_3 = env.calculate_map_at_k(linucb_arms_kb, slate_number)
curr_policy_map_at_1_per_experiment += map_at_1
curr_policy_map_at_3_per_experiment += map_at_3
curr_relevance.append(relevance_rank_at_3)

# Obtain the target analysis value
target_analysis_values = learner.get_target_analysis_value()[linucb_arms_kb[0]][
    'target_analysis_value']

In [46]:
target_analysis_values

0.8809850140905523

In [48]:
# Obtain the reward
linucb_kb_reward = env.get_online_reward_per_arm_slate(linucb_arms_kb, target_analysis_values, interaction)

# Obtain the reward for each learner
linucb_kb_regret = env.get_regret_per_arm(linucb_kb_reward, interaction)

In [49]:
linucb_kb_reward

0.8977929560602987

In [ ]:
# Update each learner's state
learner.update(arm_pulled, linucb_kb_reward, linucb_kb_regret)

test_pull_arms_top1[linucb_arms_kb[0]] += 1

for arm in linucb_arms_kb:
    test_pull_arms_top3[arm] += 1

In [7]:
for i in tqdm(range(T_test)):
    interaction = test_data.iloc[i]

    arm_pulled = mapping_MethodToArm[interaction['technique_completeness']]

    test_interaction = env.get_test_interaction(interaction["data_object"],
                                                interaction["initial_completeness"],
                                                interaction["ml_algorithm"])

    user_context = env.get_user_context_test(test_interaction)
    dataset_profiling = env.get_dataset_profiling_test(test_interaction)

    # Create user context
    learner.generate_context(user_context, dataset_profiling)

    # Select an arm for each learner
    linucb_arms_kb = learner.pull_slate_arms(slate_number)

    # Simulate the interaction
    learner.update_knowledge_base_results(interaction, dataset_profiling)

    # Calculate MAP@K
    # map_at_1 = env.calculate_map_at_1(test_interaction)
    map_at_1, relevance_rank_at_1 = env.calculate_map_at_1(linucb_arms_kb)
    map_at_3, relevance_rank_at_3 = env.calculate_map_at_k(linucb_arms_kb, slate_number)
    curr_policy_map_at_1_per_experiment += map_at_1
    curr_policy_map_at_3_per_experiment += map_at_3
    curr_relevance.append(relevance_rank_at_3)

    # Obtain the target analysis value
    target_analysis_values = learner.get_target_analysis_value()[linucb_arms_kb[0]][
        'target_analysis_value']

    # Obtain the reward
    linucb_kb_reward = env.get_online_reward_per_arm_slate(linucb_arms_kb, target_analysis_values, interaction)

    # Obtain the reward for each learner
    linucb_kb_regret = env.get_regret_per_arm(linucb_kb_reward, interaction)

    # Update each learner's state
    learner.update(arm_pulled, linucb_kb_reward, linucb_kb_regret)
    
    test_pull_arms_top1[linucb_arms_kb[0]] += 1
    
    for arm in linucb_arms_kb:
        test_pull_arms_top3[arm] += 1

  0%|          | 0/1080 [00:00<?, ?it/s]

In [8]:
test_pull_arms_top1

array([  4., 643.,   0.,   0., 239., 187.,   0.,   7.,   0.])

In [9]:
test_pull_arms_top3

array([1.300e+01, 1.069e+03, 3.000e+00, 1.000e+00, 1.059e+03, 1.072e+03,
       1.000e+00, 1.800e+01, 4.000e+00])

In [16]:
'''
"impute_standard": 112,
    "impute_mean": 115,
    "impute_median": 124,
    "impute_random": 126,
    "impute_knn": 109,
    "impute_mice": 113,
    "impute_linear_regression": 120,
    "impute_random_forest": 140,
    "impute_cmeans": 121
    '''

'\n"impute_standard": 112,\n    "impute_mean": 115,\n    "impute_median": 124,\n    "impute_random": 126,\n    "impute_knn": 109,\n    "impute_mice": 113,\n    "impute_linear_regression": 120,\n    "impute_random_forest": 140,\n    "impute_cmeans": 121\n    '

In [10]:
learner.theta

[array([ 0.        ,  0.0094942 ,  0.0495621 ,  0.25128747, -0.0874787 ,
         0.00033874]),
 array([ 0.00000000e+00,  7.10548664e-03,  1.52302863e-01,  4.24697204e-01,
         5.92645411e-01, -4.29274024e-04]),
 array([ 0.        ,  0.0013341 ,  0.01418228,  0.00308706,  0.00999737,
        -0.00020637]),
 array([ 0.00000000e+00,  1.47118707e-03,  1.39095989e-02,  7.01112801e-04,
        -7.39744373e-03, -1.84906665e-05]),
 array([ 0.        ,  0.01804917,  0.15361676,  0.52012973,  0.47691358,
        -0.00053533]),
 array([ 0.00000000e+00,  1.16074705e-02,  1.36990995e-01,  6.48965171e-01,
         3.49304537e-01, -5.29102069e-04]),
 array([ 0.00000000e+00,  1.65896071e-03,  1.29735315e-03,  4.89769146e-03,
        -1.74117152e-03, -3.46336643e-05]),
 array([ 0.        ,  0.01272721,  0.0512354 ,  0.00468338, -0.04148254,
        -0.00011951]),
 array([ 0.00000000e+00,  9.95260724e-03,  3.48070616e-02,  1.75054417e-02,
        -3.84392826e-02,  1.04085377e-05])]

In [11]:
curr_policy_map_at_1_per_experiment / len(test_data)

0.9796296296296296